In [1]:
# import the libraries
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re, string
import nltk
import spacy
import warnings
warnings.filterwarnings('ignore')

In [ ]:

train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
sample_df=pd.read_csv('sample_submission.csv')

In [ ]:
# check train_df
train_df.head()

In [ ]:
# check the shape
train_df.shape

In [ ]:
# check the missing data
train_df.isnull().sum()

In [ ]:
# check the duplicated data
train_df.duplicated().sum()

In [ ]:
# check the duplicated data
train_df['keyword'].duplicated().sum()

In [ ]:
train_df['location'].duplicated().sum()

In [ ]:
train_df['text'].duplicated().sum()

In [ ]:
# check test_df
test_df.head()

In [ ]:
# check the shape
test_df.shape

In [ ]:
# check the missing data
test_df.isnull().sum()

In [ ]:
# check the duplicated data
test_df.duplicated().sum()

In [ ]:
test_df['keyword'].duplicated().sum()

In [ ]:
test_df['location'].duplicated().sum()

In [ ]:
test_df['text'].duplicated().sum()

In [ ]:
# check test_df
test_df.head()

In [ ]:
# check the shape
test_df.shape

In [ ]:
# check the missing data
test_df.isnull().sum()

In [ ]:
# check the duplicated data
test_df.duplicated().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.pie(train_df['target'].value_counts(), labels=['Non-disaster', 'Disaster'], autopct='%0.2f')
plt.legend()  # Adds a legend
plt.title('Distribution of Disaster Tweets')  # Adds a descriptive title
plt.show()

In [ ]:
# set the random state
random_state = 4041

# import the wordcloud library
from wordcloud import WordCloud

# concat all the text for each labels
non_disaster_text = [''.join(t) for t in train_df[train_df['target']==0]['text']]
non_disaster_strings = ' '.join(map(str, non_disaster_text))
disaster_text = [''.join(t) for t in train_df[train_df['target']==1]['text']]
disaster_strings = ' '.join(map(str, disaster_text))

# generate word clouds
non_disaster_cloud = WordCloud(width=800, height=400, max_words=500, background_color='white', random_state=random_state).generate(non_disaster_strings)
disaster_cloud = WordCloud(width=800, height=400, max_words=500, random_state=random_state).generate(disaster_strings)

In [ ]:
# create subplots for the generated clouds
fig, axes = plt.subplots(1, 2, figsize = (20,20))
axes[0].imshow(non_disaster_cloud, interpolation='bilinear')
axes[1].imshow(disaster_cloud, interpolation='bilinear')

# turn the axis off
[ax.axis('off') for ax in axes]

# add titles
axes[0].set_title('Non-disaster Tweets', fontsize=16)
axes[1].set_title('Disaster Tweets', fontsize=16)

# show the figure
plt.show()